# Writing scripts for running the ChromBPNet pipeline
This notebook takes in a set of file paths and generates the necessary bash scripts to run through the ChromBPNet pipeline specified in the README file.

This notebook makes several assumptions

1. You have fragments files (BED files indicating the mapping of each read pair) for each group you want to train a model. This can be cell types, conditions, etc. or any combinations of these.
2. You have narrowPeak files for each group you want to train a model. Most of the time this should match grouping of the fragments files.
3. You have downloaded the necessary reference genome files
4. You have cloned the ChromBPNet SLURM repository to somewhere on your system
5. You have the wigToBigWig tool installed on your system
6. You have the necessary ChromBPNet conda environment set up

See the README for more details on the environment set-up

# Set-up

In [1]:
# Imports
import os
import glob
import yaml
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
# Input data paths (TODO: modify these paths to point to a directory where each group has it's own file)
path_fragments = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments"  # Directory containing fragment files
path_narrowPeaks = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks"  # Directory containing narrowPeak files

# Genome information
path_genome_fasta = "/cellar/users/aklie/data/ref/genomes/hg38/hg38.fa"
path_chromsizes = "/cellar/users/aklie/data/ref/genomes/hg38/hg38.chrom.sizes"
path_blacklist = "/cellar/users/aklie/data/ref/genomes/hg38/blacklist/blacklist.bed.gz"
path_fold_dir = "/cellar/users/aklie/data/ref/genomes/hg38/chrombpnet/splits"

# TF motif database
path_meme_file = "/cellar/users/aklie/data/ref/motifs/jvierstra/motif-clustering-v2.0beta/motifs.meme"

# Auxiliary scripts
path_chrombpnet_runner = "/cellar/users/aklie/projects/ML4GLand/chrombpnet"
path_contribution_averaging_script = f"{path_chrombpnet_runner}/scripts/average_importance_h5_over_folds.py"  # comes in the chrombpnet repo
path_pfm_script = f"{path_chrombpnet_runner}/scripts/modisco_to_pfm.py"
path_meme_script = f"{path_chrombpnet_runner}/scripts/pfm_to_meme.py"
path_wigToBigWig = "/cellar/users/aklie/opt/wigToBigWig"  # download from UCSC
path_marginalization_script = f"{path_chrombpnet_runner}/scripts/marginalization.py"
path_variant_scorer = "/cellar/users/aklie/opt/variant-scorer/src/variant_scoring.py"

# Output path
path_out = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models"
os.makedirs(path_out, exist_ok=True)

In [3]:
# Params
folds = 1  # important! TODO: modify this to the number of folds you want to train

# Find files

In [4]:
# Get a dictionary of fragments files for each group of interest (TODO: modify the extension if necessary)
fragments_dict = {}
for f in glob.glob(f"{path_fragments}/*/merged_fragments_*_sorted.tsv"):
    celltype = os.path.basename(f).split("_sorted.tsv")[0].split("fragments_")[-1]  # Extract cell type from the filename
    fragments_dict[celltype] = f
fragments_dict

{'D45_SC_delta': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D45_SC_delta/merged_fragments_D45_SC_delta_sorted.tsv',
 'D4_DE_LEFTY1+': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D4_DE_LEFTY1+/merged_fragments_D4_DE_LEFTY1+_sorted.tsv',
 'D45_early_SC_EC_beta': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D45_early_SC_EC_beta/merged_fragments_D45_early_SC_EC_beta_sorted.tsv',
 'D7_DE_SOX17+': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D7_DE_SOX17+/merged_fragments_D7_DE_SOX17+_sorted.tsv',
 'D4_DE_POLG2+': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D4_DE_POLG2+/merged_fragments_D4_DE_POLG2+_sorted.tsv',
 'D22_SC_beta': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D22_SC_beta/merged_fragments_D22_SC_beta_sorted.tsv',
 'D4_DE_MitoHi': '/cellar/users/aklie/da

In [5]:
# Get a dictionary of narrowPeak files for each group of interest (TODO: modify the extension if necessary)
narrowPeaks_dict = {}
for f in glob.glob(f"{path_narrowPeaks}/*/*.bed"):
    celltype = os.path.basename(f).split("_peaks.bed")[0].replace("_merged", "")
    if celltype in ["D15_ENP", "D15_rep1_ENP", "D15_rep2_ENP"]:
        continue
    narrowPeaks_dict[celltype] = f
narrowPeaks_dict

{'D12_liver': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks/D12_liver/D12_liver_peaks.bed',
 'D7_NECTIN3-AS1+': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks/D7_NECTIN3-AS1+/D7_NECTIN3-AS1+_peaks.bed',
 'D22_late_SC_EC_beta': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks/D22_late_SC_EC_beta/D22_late_SC_EC_beta_peaks.bed',
 'D22_late_SC_EC': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks/D22_late_SC_EC/D22_late_SC_EC_peaks.bed',
 'D12_PP_CREB5+': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks/D12_PP_CREB5+/D12_PP_CREB5+_peaks.bed',
 'D9_PFG_PLOG2+': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks/D9_PFG_PLOG2+/D9_PFG_PLOG2+_peaks.bed',
 'D45_early_SC_alpha': '/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/peaks/D45_early_SC_alpha/D45_early_SC_alpha_peaks.bed',
 'D22_late_

In [6]:
# Make sure keys are the same, if not, go back and modify the split in the celltype variable in the two previous cells
assert set(fragments_dict.keys()) == set(narrowPeaks_dict.keys())

In [7]:
# Define the groups to train models on
groups = list(fragments_dict.keys())
groups

['D45_SC_delta',
 'D4_DE_LEFTY1+',
 'D45_early_SC_EC_beta',
 'D7_DE_SOX17+',
 'D4_DE_POLG2+',
 'D22_SC_beta',
 'D4_DE_MitoHi',
 'D22_late_SC_alpha',
 'D7_GT_ONECUT1+',
 'D12_liver',
 'D15_pre_EC',
 'D45_SC_EC',
 'D15_pre_delta',
 'D4_DE_ONECUT2+',
 'D7_FLT1+',
 'D7_NECTIN3-AS1+',
 'D7_GT_CXCR4+',
 'D12_PP_ENP',
 'D45_early_others',
 'D15_PP_ERBB4+',
 'D22_late_ENP',
 'D22_late_SC_EC_beta',
 'D22_ENP_OCA2+',
 'D22_pre_alpha',
 'D9_ENP',
 'D45_early_SC_beta',
 'D15_pre_EC_beta',
 'D45_liver',
 'D15_FLT1+',
 'D12_ENP_SCG2+',
 'D4_DE_ERBB4+',
 'D12_PP_ERBB4+',
 'D22_late_SC_EC',
 'D9_NECTIN3-AS1+',
 'D45_SC_EC_beta',
 'D9_LINC01924+',
 'D4_DE_CDH8+',
 'D22_late_others',
 'D7_liver',
 'D45_SC_alpha1',
 'D15_PP_CREB5+',
 'D9_PFG_PLOG2+',
 'D12_PP_CREB5+',
 'D9_PFG_proliferating',
 'D22_SC_EC',
 'D7_GT_POLG2',
 'D45_ENP_EC',
 'D12_ENP_ARX+',
 'D9_DE_OTX2+',
 'D9_PFG_TTR+',
 'D7_GT_proliferating',
 'D45_early_SC_alpha',
 'D22_late_SC_beta',
 'D15_liver',
 'D15_pre_alpha',
 'D15_pre_beta',
 'D2

# Negatives

In [8]:
path_template = f"{path_chrombpnet_runner}/templates/negatives.txt"  # template script to generate negatives, part of this repo
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/negatives"  # output directory for scripts that will be run
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [9]:
folds = 1
celltypes = []
fragments = []
peaks = []
output_dirs = []
fold_lst = []
for group in groups:
    for i in range(folds):
        outdir = f"{path_out}/{group}/fold_{i}/negatives"
        os.makedirs(outdir, exist_ok=True)
        celltypes.append(group)
        fragments.append(fragments_dict[group])
        peaks.append(narrowPeaks_dict[group])
        output_dirs.append(outdir)
        fold_lst.append(i)
print(len(celltypes), len(fragments), len(peaks), len(output_dirs), len(fold_lst))

74 74 74 74 74


In [10]:
# Print out celltypes in same way I would write a bash array so I can copy paste into a script
celltypes_str="celltypes=(\n"
for celltype in celltypes:
    celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

# Samme for peaks
peaks_str="peaks=(\n"
for peak in peaks:
    peaks_str += f"\t{peak}\n"
peaks_str += ")"
print(peaks_str)

# Samme for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

#  Samme for fold_lst
fold_lst_str="folds=(\n"
for fold in fold_lst:
    fold_lst_str += f"\t{fold}\n"
fold_lst_str += ")"
print(fold_lst_str)

celltypes=(
	D45_SC_delta
	D4_DE_LEFTY1+
	D45_early_SC_EC_beta
	D7_DE_SOX17+
	D4_DE_POLG2+
	D22_SC_beta
	D4_DE_MitoHi
	D22_late_SC_alpha
	D7_GT_ONECUT1+
	D12_liver
	D15_pre_EC
	D45_SC_EC
	D15_pre_delta
	D4_DE_ONECUT2+
	D7_FLT1+
	D7_NECTIN3-AS1+
	D7_GT_CXCR4+
	D12_PP_ENP
	D45_early_others
	D15_PP_ERBB4+
	D22_late_ENP
	D22_late_SC_EC_beta
	D22_ENP_OCA2+
	D22_pre_alpha
	D9_ENP
	D45_early_SC_beta
	D15_pre_EC_beta
	D45_liver
	D15_FLT1+
	D12_ENP_SCG2+
	D4_DE_ERBB4+
	D12_PP_ERBB4+
	D22_late_SC_EC
	D9_NECTIN3-AS1+
	D45_SC_EC_beta
	D9_LINC01924+
	D4_DE_CDH8+
	D22_late_others
	D7_liver
	D45_SC_alpha1
	D15_PP_CREB5+
	D9_PFG_PLOG2+
	D12_PP_CREB5+
	D9_PFG_proliferating
	D22_SC_EC
	D7_GT_POLG2
	D45_ENP_EC
	D12_ENP_ARX+
	D9_DE_OTX2+
	D9_PFG_TTR+
	D7_GT_proliferating
	D45_early_SC_alpha
	D22_late_SC_beta
	D15_liver
	D15_pre_alpha
	D15_pre_beta
	D22_SC_EC_beta
	D9_PFG_CREB5+
	D22_liver
	D45_SC_alpha2
	D15_ENP_OCA2+
	D9_liver
	D22_PP_ERBB4+
	D22_SC_alpha
	D22_SC_delta+
	D4_DE_SOX4+
	D45_early_SC_EC
	D45

In [11]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run chrombpnet negatives command across multiple peaksets as a SLURM array job\n# USAGE: sbatch \\\n#--job-name=negatives \\\n#--partition carter-compute \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=16G \\\n#-n 1 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#negatives.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\n\n# file lists\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nfold=${{folds[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Peakset: $peak"\necho -e "Fold: $fold"\necho -e "Output directory: $output_dir\\n"\n\n# make the output directory\nmkdir -p $output_dir\n\n# Run cmd\ncmd="chrombpnet

In [12]:
# Write the script
with open(f"{path_scripts}/{name}_negatives.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        fold_lst_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
        path_fold_dir,
        path_blacklist
    ))

# Bias pipeline

In [13]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/bias_pipeline.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/bias_pipeline"  # output directory for scripts that will be run
name = "sc-islet-differentiation_10X-Multiome"
beta = 0.5

os.makedirs(path_scripts, exist_ok=True)

In [14]:
# Find negatives, do not run this twice without rerunning the negatives script creation above
negatives = []
for outdir in output_dirs:
    group = outdir.split("/")[-3]
    negatives.append(f"{outdir}/{group}_negatives.bed")

# Printable list
negatives_str="negatives=(\n"
for negative in negatives:
    negatives_str += f"\t{negative}\n"
negatives_str += ")"
print(negatives_str)

negatives=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/negatives/D45_SC_delta_negatives.bed
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/negatives/D4_DE_LEFTY1+_negatives.bed
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/negatives/D45_early_SC_EC_beta_negatives.bed
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/negatives/D7_DE_SOX17+_negatives.bed
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/negatives/D4_DE_POLG2+_negatives.bed
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/negatives/D22_SC_beta_negatives.bed
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/negatives/D4_DE_MitoHi_negatives.bed
	/cellar/user

In [15]:
# Update output_dirs to replace "negatives" with bias_model/$beta
output_dirs = [f"{outdir.replace('/negatives', '')}/bias_model/$beta" for outdir in output_dirs]

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_late_SC_alpha/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/sc

In [16]:
# Same for fragments
fragments_str="fragments=(\n"
for fragment in fragments:
    fragments_str += f"\t{fragment}\n"
fragments_str += ")"
print(fragments_str)

fragments=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D45_SC_delta/merged_fragments_D45_SC_delta_sorted.tsv
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D4_DE_LEFTY1+/merged_fragments_D4_DE_LEFTY1+_sorted.tsv
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D45_early_SC_EC_beta/merged_fragments_D45_early_SC_EC_beta_sorted.tsv
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D7_DE_SOX17+/merged_fragments_D7_DE_SOX17+_sorted.tsv
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D4_DE_POLG2+/merged_fragments_D4_DE_POLG2+_sorted.tsv
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D22_SC_beta/merged_fragments_D22_SC_beta_sorted.tsv
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/fragments/D4_DE_MitoHi/merged_fragments_D4_DE_MitoHi_sorted.tsv
	/cellar/user

In [17]:
# Read in template as a string
template = template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run chrombpnet bias model pipeline across multiple fragment files as a SLURM array job\n# USAGE: sbatch \\\n#--job-name=bias_pipeline \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#bias_pipeline.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\nbeta={}\n{}\n{}\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\nfragment=${{fragments[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nfold=${{folds[$SLURM_ARRAY_TASK_ID - 1]}}\nnegative=${{negatives[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLU

In [18]:
# Write the script
with open(f"{path_scripts}/{name}_bias_pipeline.sh", "w") as f:
    f.write(template.format(
        beta,
        celltypes_str,
        fragments_str,
        peaks_str,
        fold_lst_str,
        negatives_str,
        output_dirs_str,
        "ifrag",
        path_genome_fasta,
        path_chromsizes,
        path_fold_dir,
    ))


# ChromBPNet models

In [19]:
path_template = f"{path_chrombpnet_runner}/templates/chrombpnet_pipeline.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/chrombpnet_pipeline"
bias_dir = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models"  # TODO: modify this to be the bias model you want to use
beta = 0.5
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [20]:
# Find the bias models and put into matching list
bias_models_lst = []
for group in groups:
    for i in range(folds):
        bias_models_lst.append(f"{path_out}/{group}/fold_{i}/bias_model/{beta}/models/bias.h5")

# Same for bias_models
bias_models_str="bias_models=(\n"
for model in bias_models_lst:
    bias_models_str += f"\t{model}\n"
bias_models_str += ")"
print(bias_models_str)

bias_models=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/bias_model/0.5/models/bias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/bias_model/0.5/models/bias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/bias_model/0.5/models/bias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/bias_model/0.5/models/bias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/bias_model/0.5/models/bias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/bias_model/0.5/models/bias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/bias_model/0.5/models/bias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Mult

In [21]:
# Update output_dirs to be 
output_dirs = []
for group in groups:
    for i in range(folds):
        output_dirs.append(f"{path_out}/{group}/fold_{i}/chrombpnet/{beta}")

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_late_SC_alpha/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-different

In [22]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run chrombpnet model pipeline across multiple fragment files as a SLURM array job\n# USAGE: sbatch \\\n#--job-name=chrombpnet_pipeline \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#chrombpnet_pipeline.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\nbeta={}\n{}\n{}\n{}\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\nfragment=${{fragments[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nfold=${{folds[$SLURM_ARRAY_TASK_ID - 1]}}\nnegative=${{negatives[$SLURM_ARRAY_TASK_ID - 1]}}\nbias_model=${{bias_

In [23]:
# Write the script
with open(f"{path_scripts}/{name}_chrombpnet_pipeline.sh", "w") as f:
    f.write(template.format(
        beta,
        celltypes_str,
        fragments_str,
        peaks_str,
        fold_lst_str,
        negatives_str,
        bias_models_str,
        output_dirs_str,
        "ifrag",
        path_genome_fasta,
        path_chromsizes,
        path_fold_dir,
    ))

# Predictions

In [24]:
# Paths
path_template = f"{path_chrombpnet_runner}/templates/predictions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/predictions"
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [25]:
# Find bias models
bias_models = []
for outdir in output_dirs:
    #f = glob.glob(f"{outdir}/models/bias_model_scaled.h5")[0]
    f = f"{outdir}/models/bias_model_scaled.h5"
    group = outdir.split("/")[-3]
    bias_models.append(f)

# Same for bias_models
bias_models_str="bias_models=(\n"
for model in bias_models:
    bias_models_str += f"\t{model}\n"
    
bias_models_str += ")"
print(bias_models_str)

bias_models=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombpnet/0.5/model

In [26]:
# Find full chrombpnet models
chrombpnet_models = []
for outdir in output_dirs:
    #f = glob.glob(f"{outdir}//models/chrombpnet.h5")[0]
    f = f"{outdir}/models/chrombpnet.h5"
    group = outdir.split("/")[-3]
    chrombpnet_models.append(f)

# Same for chrombpnet_models
chrombpnet_models_str="chrombpnet_models=(\n"
for model in chrombpnet_models:
    chrombpnet_models_str += f"\t{model}\n"
chrombpnet_models_str += ")"
print(chrombpnet_models_str)

chrombpnet_models=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombpnet/0.5/models/chrombpnet.h5
	/cellar/users/aklie

In [27]:
# Find chrombpnet_nobias models
chrombpnet_nobias_models = []
for outdir in output_dirs:
    #f = glob.glob(f"{outdir}/models/chrombpnet_nobias.h5")[0]
    f = f"{outdir}/models/chrombpnet_nobias.h5"
    group = outdir.split("/")[-3]
    chrombpnet_nobias_models.append(f)

# Same for chrombpnet_nobias_models
chrombpnet_nobias_models_str="chrombpnet_nobias_models=(\n"
for model in chrombpnet_nobias_models:
    chrombpnet_nobias_models_str += f"\t{model}\n"
chrombpnet_nobias_models_str += ")"
print(chrombpnet_nobias_models_str)

chrombpnet_nobias_models=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombp

In [28]:
# Add predictions to output_dirs
output_dirs = [f"{outdir}/predictions" for outdir in output_dirs]

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/predictions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/predictions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/predictions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/predictions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/predictions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/predictions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombpnet/0.5/predictions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_late_

In [29]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to make predictions with a chrombpnet model\n# USAGE: sbatch \\\n#--job-name=predictions \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=32G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#predictions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\n{}\n{}\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nbias_model=${{bias_models[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_model=${{chrombpnet_models[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_nobias_model=${{chrombpnet_nobias_models[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM

In [30]:
# Write the script
with open(f"{path_scripts}/{name}_predictions.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        bias_models_str,
        chrombpnet_models_str,
        chrombpnet_nobias_models_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
    ))

# Averaging predictions

In [31]:
# Paths
path_template = f"{path_chrombpnet_runner}/templates/average_predictions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/predictions"
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [32]:
# group outdirs by celltype
groups = {}
for outdir in output_dirs:
    group = outdir.split("/")[-5]
    if group not in groups:
        groups[group] = []
    groups[group].append(outdir)
groups

{'D45_SC_delta': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/predictions'],
 'D4_DE_LEFTY1+': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/predictions'],
 'D45_early_SC_EC_beta': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/predictions'],
 'D7_DE_SOX17+': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/predictions'],
 'D4_DE_POLG2+': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/predictions'],
 'D22_SC_beta': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/predictions'],
 'D4_DE_MitoHi': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/model

In [33]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to make average prediction bigwigs over models\n# USAGE: sbatch \\\n#--job-name=average_predictions \\\n#--partition carter-compute\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=16G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#average_predictions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate eugene_tools\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib\n\n# file lists\ncelltype={}\n{}\n{}\n{}\noutput_dir={}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Bias preds: ${{bias_preds[@]}}"\necho -e "Chrombpnet nobias preds: ${{chrombpnet_nobias_preds[@]}}"\necho -e "Chrombpnet preds: ${{chrombpnet_preds[@]}}"\necho -e "Output directory: $output_dir\\n"\n\n# wigToBigWig\nwigToBigWig={}\nchromsizes={}\n\n# make the output directory\nmkdir -p $output_dir\n\n# Run cmd for bias. If there are multiple bias predictions, take the mean. Else, just copy the file\n

In [34]:
# Write the script for each celltype
for celltype, output_dirs_celltype in groups.items():
    bias_preds = [f"{outdir}/{celltype}_bias.bw" for outdir in output_dirs_celltype]
    chrombpnet_nobias_preds = [f"{outdir}/{celltype}_chrombpnet_nobias.bw" for outdir in output_dirs_celltype]
    chrombpnet_preds = [f"{outdir}/{celltype}_chrombpnet.bw" for outdir in output_dirs_celltype]
    output_dir = f"{path_out}/{celltype}/average/predictions"
    os.makedirs(output_dir, exist_ok=True)
    print(celltype)
    print(output_dir)
    bias_preds_str="bias_preds=(\n"
    for model in bias_preds:
        bias_preds_str += f"\t{model}\n"
    bias_preds_str += ")"
    print(bias_preds_str)
    chrombpnet_nobias_preds_str="chrombpnet_nobias_preds=(\n"
    for model in chrombpnet_nobias_preds:
        chrombpnet_nobias_preds_str += f"\t{model}\n"
    chrombpnet_nobias_preds_str += ")"
    print(chrombpnet_nobias_preds_str)
    chrombpnet_preds_str="chrombpnet_preds=(\n"
    for model in chrombpnet_preds:
        chrombpnet_preds_str += f"\t{model}\n"
    chrombpnet_preds_str += ")"
    print(chrombpnet_preds_str)
    with open(f"{path_scripts}/{name}_{celltype}_average_predictions.sh", "w") as f:
        f.write(template.format(
            celltype,
            bias_preds_str,
            chrombpnet_nobias_preds_str,
            chrombpnet_preds_str,
            output_dir,
            path_wigToBigWig,
            path_chromsizes
        ))

D45_SC_delta
/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/average/predictions
bias_preds=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/predictions/D45_SC_delta_bias.bw
)
chrombpnet_nobias_preds=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/predictions/D45_SC_delta_chrombpnet_nobias.bw
)
chrombpnet_preds=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/predictions/D45_SC_delta_chrombpnet.bw
)
D4_DE_LEFTY1+
/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/average/predictions
bias_preds=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/predictions/D4_DE_LEFTY1+_bias.bw
)
chrombpnet_nobias_preds=(
	/cellar/users/aklie/data/data

# Contributions

In [35]:
path_template = f"{path_chrombpnet_runner}/templates/contributions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/contributions"
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [36]:
# Replace predictions with contributions
output_dirs = [f"{outdir.replace('predictions', 'contributions')}" for outdir in output_dirs]

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/contributions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/contributions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/contributions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/contributions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/contributions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/contributions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombpnet/0.5/contributions
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/mo

In [37]:
# read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to make contributions bigwigs with one or more chrombpnet models\n# USAGE: sbatch \\\n#--job-name=contributions \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=32G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#contributions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\n{}\n{}\n{}\n{}\n\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_nobias_model=${{chrombpnet_nobias_models[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Peakset:

In [38]:
# Write the script
with open(f"{path_scripts}/{name}_contributions.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        chrombpnet_nobias_models_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
    ))

# Averaging contributions

In [39]:
path_template = f"{path_chrombpnet_runner}/templates/average_contributions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/contributions"
window=400
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [40]:
# group outdirs by celltype
groups = {}
for outdir in output_dirs:
    group = outdir.split("/")[-5]
    if group not in groups:
        groups[group] = []
    groups[group].append(outdir)
groups

{'D45_SC_delta': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/contributions'],
 'D4_DE_LEFTY1+': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/contributions'],
 'D45_early_SC_EC_beta': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/contributions'],
 'D7_DE_SOX17+': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/contributions'],
 'D4_DE_POLG2+': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/contributions'],
 'D22_SC_beta': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/contributions'],
 'D4_DE_MitoHi': ['/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Mu

In [41]:
# read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to compute averaged contributions and profiles across folds\n# USAGE: sbatch \\\n#--job-name=avg_contributions \\\n#--account=carter-compute \\\n#--output=slurm_logs/%x.%A.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#average_contributions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\\\n"\n\n# Set up environment\nsource activate eugene_tools\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib\n\n# Define input files\ncelltype={}\n{}\n{}\n{}\n{}\noutput_dir={}\nwindow={}\n\n# Create output directory\nmkdir -p $output_dir\n\n# Define paths\nwigToBigWig={}\nchromsizes={}\npath_script={}\n\n# Compute mean bigwig for counts. If there are multiple bias predictions, take the mean. Else, just copy the file\nif [ ${{#counts[@]}} -eq 1 ]; then\n    cmd="cp ${{counts[0]}} $output_dir/${{celltype}}_counts.bw"\nelse\n    cmd="wiggletools mean ${{counts[@]}} > ${{celltype}}_temp_counts.wig && $wigToBigWig $

In [42]:
# Write the script for each celltype
for celltype, output_dirs_celltype in groups.items():
    counts = [f"{outdir}/{celltype}.counts_scores.bw" for outdir in output_dirs_celltype]
    profile = [f"{outdir}/{celltype}.profile_scores.bw" for outdir in output_dirs_celltype]
    counts_h5 = [f"{outdir}/{celltype}.counts_scores.h5" for outdir in output_dirs_celltype]
    profile_h5 = [f"{outdir}/{celltype}.profile_scores.h5" for outdir in output_dirs_celltype]
    output_dir = f"{path_out}/{celltype}/average/contributions"
    os.makedirs(output_dir, exist_ok=True)

    counts_str="counts=(\n"
    for model in counts:
        counts_str += f"\t{model}\n"
    counts_str += ")"
    print(counts_str)

    profile_str="profile=(\n"
    for model in profile:
        profile_str += f"\t{model}\n"
    profile_str += ")"
    print(profile_str)

    counts_h5_str="counts_h5=(\n"
    for model in counts_h5:
        counts_h5_str += f"\t{model}\n"
    counts_h5_str += ")"
    print(counts_h5_str)

    profile_h5_str="profile_h5=(\n"
    for model in profile_h5:
        profile_h5_str += f"\t{model}\n"
    profile_h5_str += ")"
    print(profile_h5_str)

    with open(f"{path_scripts}/{name}_{celltype}_average_contributions.sh", "w") as f:
        f.write(template.format(
            celltype,
            counts_str,
            profile_str,
            counts_h5_str,
            profile_h5_str,
            output_dir,
            window,
            path_wigToBigWig,
            path_chromsizes,
            path_contribution_averaging_script,
        ))

counts=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/contributions/D45_SC_delta.counts_scores.bw
)
profile=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/contributions/D45_SC_delta.profile_scores.bw
)
counts_h5=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/contributions/D45_SC_delta.counts_scores.h5
)
profile_h5=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/contributions/D45_SC_delta.profile_scores.h5
)
counts=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/contributions/D4_DE_LEFTY1+.counts_scores.bw
)
profile=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/contribution

# De novo motif discovery

In [43]:
path_template = f"{path_chrombpnet_runner}/templates/motif_discovery.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/motifs"
n_seqlets = 100000
leiden_res = 2
window = 400
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [44]:
# read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run de novo motif discovery on contributions\n# USAGE: sbatch \\\n#--job-name=motif_discovery \\\n#--partition=carter-compute \\\n#--output=slurm_logs/%x.%A.%a.out \\\n#--array=1-12%12 \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#motif_discovery.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set up environment\nsource activate eugene_tools\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/opt/miniconda3/lib/\nexport LD_LIBRARY_PATH=/cm/shared/apps/slurm/current/lib64\nNUMBA_NUM_THREADS=$SLURM_CPUS_PER_TASK\n\n# Define input files\n{}\n{}\n{}\n{}\nn_seqlets={}\nleiden_res={}\nwindow={}\n\n# Make the fold the slurm task id - 1\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\ninput_h5=${{input_h5s[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\ntype=${{types[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# Create output directory\nmkdir -p $output_dir\n\n# Run motifs cmd\ncmd="modisco motifs \\\n-i $input_h5 \\\n-n 

In [45]:
# Find each cell types averaged contributions
avg_contributions = []
for group in groups:
    avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_counts.h5")
    #avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_profile.h5")

avg_contributions_str="input_h5s=(\n"
for model in avg_contributions:
    avg_contributions_str += f"\t{model}\n"
avg_contributions_str += ")"
print(avg_contributions_str)

input_h5s=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/average/contributions/D45_SC_delta_counts.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/average/contributions/D4_DE_LEFTY1+_counts.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/average/contributions/D45_early_SC_EC_beta_counts.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/average/contributions/D7_DE_SOX17+_counts.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/average/contributions/D4_DE_POLG2+_counts.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/average/contributions/D22_SC_beta_counts.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/average/contributions/D4_DE_MitoHi_counts.h5
	/cell

In [46]:
# celltypes x2
celltypes_str="celltypes=(\n"
for celltype in groups.keys():
    celltypes_str += f"\t{celltype}\n"
    #celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

celltypes=(
	D45_SC_delta
	D4_DE_LEFTY1+
	D45_early_SC_EC_beta
	D7_DE_SOX17+
	D4_DE_POLG2+
	D22_SC_beta
	D4_DE_MitoHi
	D22_late_SC_alpha
	D7_GT_ONECUT1+
	D12_liver
	D15_pre_EC
	D45_SC_EC
	D15_pre_delta
	D4_DE_ONECUT2+
	D7_FLT1+
	D7_NECTIN3-AS1+
	D7_GT_CXCR4+
	D12_PP_ENP
	D45_early_others
	D15_PP_ERBB4+
	D22_late_ENP
	D22_late_SC_EC_beta
	D22_ENP_OCA2+
	D22_pre_alpha
	D9_ENP
	D45_early_SC_beta
	D15_pre_EC_beta
	D45_liver
	D15_FLT1+
	D12_ENP_SCG2+
	D4_DE_ERBB4+
	D12_PP_ERBB4+
	D22_late_SC_EC
	D9_NECTIN3-AS1+
	D45_SC_EC_beta
	D9_LINC01924+
	D4_DE_CDH8+
	D22_late_others
	D7_liver
	D45_SC_alpha1
	D15_PP_CREB5+
	D9_PFG_PLOG2+
	D12_PP_CREB5+
	D9_PFG_proliferating
	D22_SC_EC
	D7_GT_POLG2
	D45_ENP_EC
	D12_ENP_ARX+
	D9_DE_OTX2+
	D9_PFG_TTR+
	D7_GT_proliferating
	D45_early_SC_alpha
	D22_late_SC_beta
	D15_liver
	D15_pre_alpha
	D15_pre_beta
	D22_SC_EC_beta
	D9_PFG_CREB5+
	D22_liver
	D45_SC_alpha2
	D15_ENP_OCA2+
	D9_liver
	D22_PP_ERBB4+
	D22_SC_alpha
	D22_SC_delta+
	D4_DE_SOX4+
	D45_early_SC_EC
	D45

In [47]:
# Make the output dirs x 2 
output_dirs = []
for group in groups:
    output_dirs.append(f"{path_out}/{group}/average/motifs")

output_dirs_str="output_dirs=(\n"
for model in output_dirs:
    output_dirs_str += f"\t{model}\n"
    #output_dirs_str += f"\t{model}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_late_SC_alpha/average/motifs
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_GT_ONECUT1+/average/motifs

In [48]:
# get one of each of type: counts and profile
types = []
for group in groups:
    types.append("counts")
    #types.append("profile")
types_str="types=(\n"
for model in types:
    types_str += f"\t{model}\n"
types_str += ")"
print(types_str)

types=(
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
	counts
)


In [49]:
# Write the script
with open(f"{path_scripts}/{name}_motif_discovery.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        avg_contributions_str,
        output_dirs_str,
        types_str,
        n_seqlets,
        leiden_res,
        window,
        path_meme_file,
        path_pfm_script,
    ))

# Motif clustering

In [50]:
path_template = f"{path_chrombpnet_runner}/templates/motif_clustering.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/motifs"
os.makedirs(path_scripts, exist_ok=True)
t = 0.999

In [58]:
# Find paths to pfms
paths_pfm = []
for group in groups:
    #paths_pfm.append(f"{path_out}/{group}/average/motifs/pfms/counts/{group}.counts.pfm")
    #paths_pfm.append(f"{path_out}/{group}/average/motifs/pfms/profile/{group}.profile.pfm")
    # if there is a counts pfm, include it
    f = glob.glob(f"{path_out}/{group}/average/motifs/pfms/counts/*.counts.pfm")
    if len(f) > 0:
        paths_pfm.append(f[0])
paths_pfm_str="paths_pfm=(\n"
for model in paths_pfm:
    paths_pfm_str += f"\t{model}\n"
paths_pfm_str += ")"
print(paths_pfm_str)

paths_pfm=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/average/motifs/pfms/counts/D45_SC_delta.counts.pfm
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/average/motifs/pfms/counts/D4_DE_LEFTY1+.counts.pfm
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/average/motifs/pfms/counts/D45_early_SC_EC_beta.counts.pfm
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/average/motifs/pfms/counts/D7_DE_SOX17+.counts.pfm
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/average/motifs/pfms/counts/D4_DE_POLG2+.counts.pfm
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/average/motifs/pfms/counts/D22_SC_beta.counts.pfm
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/average/motifs/

In [59]:
len(paths_pfm)

74

In [55]:
# Read in template as a string
template = open(path_template).read()

In [56]:
# Write the script
with open(f"{path_scripts}/{name}_motif_clustering.sh", "w") as f:
    f.write(template.format(
        paths_pfm_str,
        path_out + "/motifs",
        t,
        path_meme_script,
        path_meme_file
    ))

# Motif hit calling

In [175]:
path_template = f"{path_chrombpnet_runner}/templates/motif_hits.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/motifs"
os.makedirs(path_scripts, exist_ok=True)

In [176]:
# Cell types
celltypes_str="celltypes=(\n"
for celltype in groups.keys():
    celltypes_str += f"\t{celltype}\n"
    #celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

# Find peaks corresponding to modisco celltypes
peaks = []
for group in groups:
    peaks.append(narrowPeaks_dict[group])
    #peaks.append(narrowPeaks_dict[group])
peaks_str="peaks=(\n"
for model in peaks:
    peaks_str += f"\t{model}\n"
peaks_str += ")"
print(peaks_str)

# Find each cell types averaged contributions
avg_contributions = []
for group in groups:
    avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_counts.h5")
    #avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_profile.h5")
avg_contributions_str="input_h5s=(\n"
for model in avg_contributions:
    avg_contributions_str += f"\t{model}\n"
avg_contributions_str += ")"
print(avg_contributions_str)

# Find paths to modiscos
modiscos = []
for group in groups:
    modiscos.append(f"{path_out}/{group}/average/motifs/{group}.counts.modisco.h5")
    #modiscos.append(f"{path_out}/{group}/average/motifs/{group}.profile.modisco.h5")
modiscos_str="modiscos=(\n"
for model in modiscos:
    modiscos_str += f"\t{model}\n"
modiscos_str += ")"
print(modiscos_str)

# types
types = []
for group in groups:
    types.append("counts")
    #types.append("profile")
types_str="types=(\n"
for model in types:
    types_str += f"\t{model}\n"
types_str += ")"
print(types_str)

# output_dirs
output_dirs = []
for group in groups:
    output_dirs.append(f"{path_out}/{group}/average/motifs/hits/counts")
    #output_dirs.append(f"{path_out}/{group}/average/motifs/hits/profile")
output_dirs_str="output_dirs=(\n"
for model in output_dirs:
    output_dirs_str += f"\t{model}\n"
output_dirs_str += ")"
print(output_dirs_str)

celltypes=(
	D45_SC_delta
	D4_DE_LEFTY1+
	D45_early_SC_EC_beta
	D7_DE_SOX17+
	D4_DE_POLG2+
	D22_SC_beta
	D4_DE_MitoHi
	D22_late_SC_alpha
	D7_GT_ONECUT1+
	D12_liver
	D15_pre_EC
	D45_SC_EC
	D15_pre_delta
	D4_DE_ONECUT2+
	D7_FLT1+
	D7_NECTIN3-AS1+
	D7_GT_CXCR4+
	D12_PP_ENP
	D45_early_others
	D15_PP_ERBB4+
	D22_late_ENP
	D22_late_SC_EC_beta
	D22_ENP_OCA2+
	D22_pre_alpha
	D9_ENP
	D45_early_SC_beta
	D15_pre_EC_beta
	D45_liver
	D15_FLT1+
	D12_ENP_SCG2+
	D4_DE_ERBB4+
	D12_PP_ERBB4+
	D22_late_SC_EC
	D9_NECTIN3-AS1+
	D45_SC_EC_beta
	D9_LINC01924+
	D4_DE_CDH8+
	D22_late_others
	D7_liver
	D45_SC_alpha1
	D15_PP_CREB5+
	D9_PFG_PLOG2+
	D12_PP_CREB5+
	D9_PFG_proliferating
	D22_SC_EC
	D7_GT_POLG2
	D45_ENP_EC
	D12_ENP_ARX+
	D9_DE_OTX2+
	D9_PFG_TTR+
	D7_GT_proliferating
	D45_early_SC_alpha
	D22_late_SC_beta
	D15_liver
	D15_pre_alpha
	D15_pre_beta
	D22_SC_EC_beta
	D9_PFG_CREB5+
	D22_liver
	D45_SC_alpha2
	D15_ENP_OCA2+
	D9_liver
	D22_PP_ERBB4+
	D22_SC_alpha
	D22_SC_delta+
	D4_DE_SOX4+
	D45_early_SC_EC
	D45

In [177]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run motif hit calling\n# USAGE: sbatch \\\n#--job-name=motif_hits \\\n#--partition=carter-gpu \\\n#--account=carter-gpu \\\n#--gpus=a30:1 \\\n#--output=slurm_logs/%x.%A.%a.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#motif_hits.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Configure environment\nsource activate finemo_gpu\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/opt/miniconda3/envs/finemo_gpu/lib/\n\n# Define input files\n{}\n{}\n{}\n{}\n{}\n{}\nwindow={}\n\n# Make the fold the slurm task id - 1\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\ninput_h5=${{input_h5s[$SLURM_ARRAY_TASK_ID - 1]}}\nmodisco=${{modiscos[$SLURM_ARRAY_TASK_ID - 1]}}\ntype=${{types[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# Create output directory\nmkdir -p $output_dir\n\n# Run cmd to preprocess\ncmd="finemo extract-regions-chrombpnet-h5 

In [178]:
# Write the script
with open(f"{path_scripts}/{name}_motif_hits.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        avg_contributions_str,
        modiscos_str,
        types_str,
        output_dirs_str,
        window,
    ))

# Motif marginalization

In [179]:
path_template = f"{path_chrombpnet_runner}/templates/motif_marginalization.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/motifs"
n_seqs = 100
os.makedirs(path_scripts, exist_ok=True)

In [180]:
path_denovo_motifs = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/motifs/meme/combined.meme"

In [181]:
# Update output_dirs to be 
output_dirs = []
for group in groups:
    if "_dex" in group:
        continue
    for i in range(folds):
        output_dirs.append(f"{path_out}/{group}/fold_{i}/chrombpnet/{beta}/marginalization/")

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

# Find full chrombpnet models
chrombpnet_models = []
for outdir in output_dirs:
    f = outdir.replace("marginalization/", "models/chrombpnet_nobias.h5")
    chrombpnet_models.append(f)

# Same for chrombpnet_models
chrombpnet_models_str="models=(\n"
for model in chrombpnet_models:
    chrombpnet_models_str += f"\t{model}\n"
chrombpnet_models_str += ")"
print(chrombpnet_models_str)

# Find peaks corresponding to modisco celltypes
peaks = []
for group in groups:
    if "_dex" in group:
        continue
    peaks.append(narrowPeaks_dict[group])
peaks_str="peaks=(\n"
for model in peaks:
    peaks_str += f"\t{model}\n"
peaks_str += ")"
print(peaks_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/marginalization/
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/marginalization/
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/marginalization/
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/marginalization/
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/marginalization/
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/marginalization/
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombpnet/0.5/marginalization/
	/cellar/users/aklie/data/datasets/sc-islet-differenti

In [182]:
# Read in template as a string
template = open(path_template).read()
template

'#!/bin/bash\n\n#####\n# Script to run marginalization analysis with chromBPNet models\n# USAGE:\n# sbatch \\\n# --job-name=marginalize \\\n# --account carter-gpu \\\n# --partition carter-gpu \\\n# --gpus=a30:1 \\\n# --output slurm_logs/%x.%A.%a.out \\\n# --mem=32G \\\n# -n 4 \\\n# -t 02-00:00:00 \\\n# --array=1-20%10 \\\n# marginalize.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate eugene_tools\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import torch; print(torch.cuda.is_available())"\n\n# File arrays\n{}\n{}\n{}\nn_seqs={}\n\n# SLURM task-specific values\nmodel=${{models[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# Constants\ngenome_fasta="{}"\nmotif_file="{}"\n\necho -e "Model: $model"\necho -e "Peaks: $peak"\necho -e "Output: $output_dir\\n"\n\nmkdir -p $output_dir\n\n# Run marginalization\ncmd="

In [183]:
# Write the script
with open(f"{path_scripts}/{name}_motif_marginalization.sh", "w") as f:
    f.write(template.format(
        chrombpnet_models_str,
        peaks_str,
        output_dirs_str,
        n_seqs,
        path_genome_fasta,
        path_denovo_motifs,
        path_marginalization_script,
    ))

# Variant scoring

In [184]:
# Paths
path_template = f"{path_chrombpnet_runner}/templates/variant_scoring.txt"
path_scripts = "/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/bin/chrombpnet/scripts/variant_scoring"    
paths_variants = glob.glob("/cellar/users/aklie/data/ref/variants/*/*.bed")
# Keep only if T1D and T2D in the path
paths_variants = [x for x in paths_variants if ("T1D" in x) or ("T2D" in x)]
paths_variants = [x for x in paths_variants if "chrombpnet" in x]
paths_variants.pop(1)
names_variants = ["T1D_Chiou_2021", "T2D_DIAMANTE_multiancestry"]
name = "sc-islet-differentiation_10X-Multiome"

os.makedirs(path_scripts, exist_ok=True)

In [185]:
paths_variants, names_variants

(['/cellar/users/aklie/data/ref/variants/T1D/T1D_Chiou_2021_cred_set.wGWAS_info.minimal.snps.chrombpnet.bed',
  '/cellar/users/aklie/data/ref/variants/T2D/T2D_DIAMANTE_multiancestry.cred99.hg38.wGWAS_info.minimal.snps.chrombpnet.bed'],
 ['T1D_Chiou_2021', 'T2D_DIAMANTE_multiancestry'])

In [186]:
# Update output_dirs to be 
output_dirs = []
for group in groups:
    for i in range(folds):
        output_dirs.append(f"{path_out}/{group}/fold_{i}/chrombpnet/{beta}")

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_late_SC_alpha/fold_0/chrombpnet/0.5
	/cellar/users/aklie/data/datasets/sc-islet-different

In [188]:
outdir

'/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_GT_ONECUT1+/fold_0/chrombpnet/0.5'

In [189]:
# Find chrombpnet_nobias models
chrombpnet_nobias_models = []
for outdir in output_dirs:
    #f = glob.glob(f"{outdir}/models/chrombpnet_nobias.h5")[0]
    f = f"{outdir}/models/chrombpnet_nobias.h5"
    group = outdir.split("/")[-3]
    chrombpnet_nobias_models.append(f)

# Same for chrombpnet_nobias_models
chrombpnet_nobias_models_str="chrombpnet_nobias_models=(\n"
for model in chrombpnet_nobias_models:
    chrombpnet_nobias_models_str += f"\t{model}\n"
chrombpnet_nobias_models_str += ")"
print(chrombpnet_nobias_models_str)

chrombpnet_nobias_models=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/chrombp

In [190]:
# Define the groups to train models on
groups = list(fragments_dict.keys())

celltypes_str="celltypes=(\n"
for celltype in groups:
    for i in range(folds):
        celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

celltypes=(
	D45_SC_delta
	D4_DE_LEFTY1+
	D45_early_SC_EC_beta
	D7_DE_SOX17+
	D4_DE_POLG2+
	D22_SC_beta
	D4_DE_MitoHi
	D22_late_SC_alpha
	D7_GT_ONECUT1+
	D12_liver
	D15_pre_EC
	D45_SC_EC
	D15_pre_delta
	D4_DE_ONECUT2+
	D7_FLT1+
	D7_NECTIN3-AS1+
	D7_GT_CXCR4+
	D12_PP_ENP
	D45_early_others
	D15_PP_ERBB4+
	D22_late_ENP
	D22_late_SC_EC_beta
	D22_ENP_OCA2+
	D22_pre_alpha
	D9_ENP
	D45_early_SC_beta
	D15_pre_EC_beta
	D45_liver
	D15_FLT1+
	D12_ENP_SCG2+
	D4_DE_ERBB4+
	D12_PP_ERBB4+
	D22_late_SC_EC
	D9_NECTIN3-AS1+
	D45_SC_EC_beta
	D9_LINC01924+
	D4_DE_CDH8+
	D22_late_others
	D7_liver
	D45_SC_alpha1
	D15_PP_CREB5+
	D9_PFG_PLOG2+
	D12_PP_CREB5+
	D9_PFG_proliferating
	D22_SC_EC
	D7_GT_POLG2
	D45_ENP_EC
	D12_ENP_ARX+
	D9_DE_OTX2+
	D9_PFG_TTR+
	D7_GT_proliferating
	D45_early_SC_alpha
	D22_late_SC_beta
	D15_liver
	D15_pre_alpha
	D15_pre_beta
	D22_SC_EC_beta
	D9_PFG_CREB5+
	D22_liver
	D45_SC_alpha2
	D15_ENP_OCA2+
	D9_liver
	D22_PP_ERBB4+
	D22_SC_alpha
	D22_SC_delta+
	D4_DE_SOX4+
	D45_early_SC_EC
	D45

In [191]:
# Get template
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to get variant scores with a trained chrombpnet model\n# USAGE: sbatch \\\n#--job-name=variant_scoring \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=32G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#variant_scoring.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\n{}\n{}\n{}\nlist={}\nrandom_seed={}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_nobias_model=${{chrombpnet_nobias_models[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Model path: $chrombpnet_nobias_model"\n

In [192]:
# for each variant list
for path_variant, name_variant in zip(paths_variants, names_variants):

    # Output dirs are the same as the chrombpnet_nobias models
    output_dirs_str="output_dirs=(\n"
    for output_dir in output_dirs:
        output_dirs_str += f"\t{output_dir}/variant_scoring/{name_variant}\n"
    output_dirs_str += ")"
    print(output_dirs_str)

    # Write the script
    with open(f"{path_scripts}/{name}_{name_variant}_variant_scoring.sh", "w") as f:
        f.write(template.format(
            celltypes_str,
            chrombpnet_nobias_models_str,
            output_dirs_str,
            path_variant,
            1234,
            path_variant_scorer,
            path_genome_fasta,
            path_chromsizes,
        ))

output_dirs=(
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_SC_delta/fold_0/chrombpnet/0.5/variant_scoring/T1D_Chiou_2021
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_LEFTY1+/fold_0/chrombpnet/0.5/variant_scoring/T1D_Chiou_2021
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D45_early_SC_EC_beta/fold_0/chrombpnet/0.5/variant_scoring/T1D_Chiou_2021
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D7_DE_SOX17+/fold_0/chrombpnet/0.5/variant_scoring/T1D_Chiou_2021
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_POLG2+/fold_0/chrombpnet/0.5/variant_scoring/T1D_Chiou_2021
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D22_SC_beta/fold_0/chrombpnet/0.5/variant_scoring/T1D_Chiou_2021
	/cellar/users/aklie/data/datasets/sc-islet-differentiation_10X-Multiome/models/D4_DE_MitoHi/fold_0/ch

# DONE!

---